### Prediction script:
#### Following program serves following purposes:
* Iteratively detects test images from a local folder path
* Loads food model trained previously for 16 classes. 
* Performs preprocessing on incoming images and parse them through the model to ouput predicted lables.

In [1]:
import numpy as np
import pandas as pd
import os

#### training images from local machine

In [2]:
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.utils import np_utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [85]:
'''def process_whole(img):#takes in array of image paths and outputs array of image matrix
    image_array = np.empty((img.shape[0],120,120,3))
    for i in range(img.shape[0]):
        a= load_img(img[i], target_size=(120,120))#(180,260))
        image_array[i,:,:,:]= img_to_array(a)
    return image_array 
  
def test_process(img):
    a= load_img(img, target_size= (120,120))
    return img_to_array(a)'''

In [3]:
from keras.models import load_model #loading a pretrained model
model16c= load_model('C:\\model16c_91.75.hdf5')#loading a model pretrained model for 16 classes from local path
print(model16c.inputs)

[<tf.Tensor 'input_1:0' shape=(?, 180, 180, 3) dtype=float32>]


* #### Auxilliary functions to preprocess test images and throw predition array, top prediction.

In [6]:
def process_whole(img):#takes in array of image paths and outputs array of image matrix
    image_array = np.empty((img.shape[0],180,180,3))
    for i in range(img.shape[0]):#here img is an array holding image paths, shaped(num_images, 1)  
        a= load_img(img[i], target_size=(180,180))#(180,260))
        image_array[i,:,:,:]= img_to_array(a)
    return image_array 

In [ ]:
def chkname(idx):
    print('probabilities: ', pred_array[idx], '\n \n max probability pos at: ', np.argmax(pred_array[idx]),
          '\n \n val of probabilty: ',pred_array[idx][np.argmax(pred_array[idx])])

* ##### Following appends incoming images into a testmix list and preprocesses it to 180x180x3 matrices

In [7]:
li_testmix=[]#list of mixed images from test folder

for file in os.scandir('C:\\tt23'):#traverses over any image thrown in this folder
    li_testmix.append(file.path)#os.path.join(root, file))

test_patharray= np.array(li_testmix)
print(test_patharray.shape, test_patharray)

test_array = process_whole(test_patharray) #array of image matrix
print(test_array.shape)

(4,) ['C:\\tt23\\25.jpg' 'C:\\tt23\\26.jpg' 'C:\\tt23\\27.jpg'
 'C:\\tt23\\28.jpg']
(4, 180, 180, 3)


In [10]:
import time
tick = time.time()
pred_array = model16c.predict(test_array)####testing model predictions on test dataset

print('exe time: ', time.time() -tick)

exe time:  4.699268579483032


In [135]:
chkname(3)

probabilities:  [1.9544427e-06 2.4477117e-07 1.8792628e-06 2.4906598e-07 6.8006005e-07
 4.4285494e-07 1.9865936e-08 3.9805968e-06 1.0443083e-05 9.9992883e-01
 5.4473645e-07 1.3718718e-05 3.5610024e-05 1.7599281e-07 1.2484621e-06
 2.7302773e-08] 
 
 max probability pos at:  9 
 
 val of probabilty:  0.99992883


##### Following dictionary holds class labels as keys and food names, GI index as values.
* It serves as reference to throw a food name & GI index corresponding to a numeric class label.

In [22]:
class_gi_dict = {0:['Samosa', 'GI: 75 to 85'], 1:['Idly','GI: 60 to 80'], 2:['ice_cream', 'GI: 85 to 95' ], 3:['dosa','GI: 61 to 85'], 4:['hamburger','GI: 56 to 58'],5:['cup_cake','GI: 56 to 58'], 6:['donuts','GI: 56 to 58'],7:['Omelette','GI: 60 to 75'],8:['chicken_wings','GI: 90 to 100'],9:['french_fries','GI: 75 to 95'], 10:['dumplings','GI: 65 to 85'], 11:['hotdog','GI: 80 to 95'], 12:['fried_rice','GI: 60 to 70'], 13:['waffles','GI: 30 to 40'], 14:['pizza','GI: 85 to 98'], 15:['sandwich','GI: 30 to 40']}

##### evaluating all food name & Indexes predicted against random number of images thrown in testmix folder

In [27]:
#pred_array = model16c.predict(test_array)####testing model predictions on test dataset

#for file in test_iter:
for i, file in enumerate(os.scandir('C:\\tt23')):
    #print('cake is [1 0 0 0], dosa is [0 1 0 0], poori is [0 0 1 0] , dosa is [0 0 0 1]')
    print('image named: [{a}], \n yields prediction array: [{b}]'.format(a= file.name, b= pred_array[i]))
    print('\n predicted food in image: [{a}] is: [{b}] with the probability of: [{c}] and GI index: [{d}]'.format(a= file.name, b= class_gi_dict[np.argmax(pred_array[i])][0], c= np.amax(pred_array[i])*100, d= class_gi_dict[np.argmax(pred_array[i])][1]), '\n \n')


image named: [25.jpg], 
 yields prediction array: [[2.6665629e-07 8.5585384e-08 4.4159756e-07 7.3102790e-08 1.7772120e-07
 2.4154852e-07 4.4824895e-07 6.9917383e-05 4.5211414e-07 5.0491457e-07
 1.1032588e-06 6.8183233e-08 8.9386123e-07 2.9987922e-07 9.9992466e-01
 3.2791124e-07]]

 predicted food in image: [25.jpg] is: [pizza] with the probability of: [99.99246597290039] and GI index: [GI: 85 to 98] 
 

image named: [26.jpg], 
 yields prediction array: [[3.4190273e-06 1.0983578e-04 3.9239058e-06 9.7042680e-01 4.9790838e-06
 6.1200103e-06 8.8558627e-05 7.7617001e-03 8.5718821e-06 1.2097621e-05
 4.5439606e-06 2.7032856e-06 5.2052789e-04 7.6342229e-05 2.0945990e-02
 2.3833027e-05]]

 predicted food in image: [26.jpg] is: [dosa] with the probability of: [97.04267978668213] and GI index: [GI: 61 to 85] 
 

image named: [27.jpg], 
 yields prediction array: [[9.99999046e-01 1.09474296e-08 2.15736740e-09 1.08366127e-09
 2.44970177e-08 3.35703215e-10 1.32634383e-07 2.85043939e-07
 5.44695455e-1